# Task 1: Human recognition using ImageAI

In [1]:
# Area for modules installation


In [75]:
# Importing required modules and setting up environment
import warnings
warnings.filterwarnings('ignore')
from imageai.Detection import ObjectDetection
from IPython.display import Image
import pandas as pd

In [ ]:
# Function for counting of number of found people on an image
def human_counter(image_data):
    counter = 0
    for record in image_data:
        if record['name'] == 'person':
            counter+=1
    return counter

In [54]:
# Paths to images
models_path = 'models/'
image_path = 'resources/images/'
image_processed_path = 'resources/images_processed/'

# Getting list of available images
images_available = os.listdir(image_path)
print(images_available)

['2.jpg', '7.jpg', '5.jpg', '4.jpg', '6.jpg', '10.jpg', '3.jpg', '1.jpg', '9.jpg', '8.jpg']


In [55]:
# Setting up RetinaNet detector
detector_RN = ObjectDetection()
detector_RN.setModelTypeAsRetinaNet() 
detector_RN.setModelPath(models_path + 'resnet50_coco_best_v2.0.1.h5') 
detector_RN.loadModel()

# Setting up YOLO detector
detector_YOLO = ObjectDetection()
detector_YOLO.setModelTypeAsYOLOv3() 
detector_YOLO.setModelPath(models_path + 'yolo.h5')  
detector_YOLO.loadModel()

# Objects detection threshold is 50

In [79]:
# Array of true number of people on every image
people_true_number = [4, 5, 7, 4, 4, 5, 6, 4, 5, 6]

In [81]:
results_array = []
#results_array.append(['image/model', 'RetinaNet50', 'RetinaNet80', 'YOLO50', 'YOLO80'])

for image_number in range(1,10+1):
    
    # Initializing row array
    temp_array = []

    # Adding image number
    temp_array.append(image_number)
    
    # Adding info about real number of people on the image
    temp_array.append(people_true_number[image_number-1])

    # Using RetinaNet with 50% cutoff
    image_data = detector_RN.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_RN_Cutoff_50' + '.jpg',
    minimum_percentage_probability = 50 # Cutoff treshold
    )
    temp_array.append(human_counter(image_data))
    
    # Using RetinaNet with 80% cutoff
    image_data = detector_RN.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_RN_Cutoff_80' + '.jpg',
    minimum_percentage_probability = 80 # Cutoff treshold
    )
    temp_array.append(human_counter(image_data))
    
    # Using YOLO with 50% cutoff
    image_data = detector_RN.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_YOLO_Cutoff_50' + '.jpg',
    minimum_percentage_probability = 50 # Cutoff treshold
    )
    temp_array.append(human_counter(image_data))
    
    # Using YOLO with 80% cutoff
    image_data = detector_RN.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_YOLO_Cutoff_80' + '.jpg',
    minimum_percentage_probability = 80 # Cutoff treshold
    )
    temp_array.append(human_counter(image_data))
    
    # Adding entire row into resulting array
    results_array.append(temp_array)
    #Image(filename = (image_path + str(image_number) + '.jpg'))

In [89]:
# Data visualization
table_columns_names = ['Image number', 'True value', 'RetinaNet50', 'RetinaNet80', 'YOLO50', 'YOLO80']
pd.DataFrame(results_array, columns = table_columns_names)

,Image number,True value,RetinaNet50,RetinaNet80,YOLO50,YOLO80
0,1,4,5,4,5,4
1,2,5,6,1,6,1
2,3,7,5,2,5,2
3,4,4,4,2,4,2
4,5,4,5,4,5,4
5,6,5,5,4,5,4
6,7,6,6,6,6,6
7,8,4,4,4,4,4
8,9,5,5,5,5,5
9,10,6,5,3,5,3


In [86]:
model_results_array = []
for i in range(0, 9+1):
    # Getting current values
    true_value = results_array[i][model_type]
    predicted_value = results_array[i][model_type]
    
    # Distributing obtained results
    if true_value >= predicted_value:
        TP = predicted_value
        TN = 0
        FP = 0
        FN = true_value - predicted_value
    else:
        TP = true_value
        TN = 0
        FP = predicted_value - true_value
        FN = 0
    
    # Adding metrics to array
    model_results_array.append([TP, TN, FP, FN])

print(model_results_array)

[[4, 0, 1, 0], [5, 0, 1, 0], [5, 0, 0, 2], [4, 0, 0, 0], [4, 0, 1, 0], [5, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [5, 0, 0, 1]]


In [90]:
# Creating dictionary for model's result storage
all_results = {}

for model_type in range(2, 5+1):
    
    # Arranging array for model resutls
    model_results_array = []
    
    # Iterating through rows
    for i in range(0, 9+1):
        
        # Getting current values
        true_value = results_array[i][model_type]
        predicted_value = results_array[i][model_type]
    
        # Distributing obtained results
        if true_value >= predicted_value:
            TP = predicted_value
            TN = 0
            FP = 0
            FN = true_value - predicted_value
        else:
            TP = true_value
            TN = 0
            FP = predicted_value - true_value
            FN = 0
    
        # Adding metrics to array
        model_results_array.append([TP, TN, FP, FN])
        
    all_results.update({table_columns_names[model_type]: model_results_array})
    
print(all_results)

{'RetinaNet50': [[5, 0, 0, 0], [6, 0, 0, 0], [5, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [5, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [5, 0, 0, 0]], 'RetinaNet80': [[4, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [2, 0, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [3, 0, 0, 0]], 'YOLO50': [[5, 0, 0, 0], [6, 0, 0, 0], [5, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [5, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [5, 0, 0, 0]], 'YOLO80': [[4, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [2, 0, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [3, 0, 0, 0]]}
